In [1]:
# 数据集中彩色的掩码，按照colors.json中给出的编号，替换为classid写入到单通道掩码图片中
import os
import cv2
import numpy as np
import json
import glob
import matplotlib.pyplot as plt

h, w = 1024, 1280

In [5]:
with open("colors.json", "r") as f:
    label_list = json.load(f)
# label_list = label_list[:8]
label_list


[{'name': 'background-tissue', 'color': [0, 0, 0], 'classid': 0},
 {'name': 'instrument-shaft', 'color': [0, 255, 0], 'classid': 1},
 {'name': 'instrument-clasper', 'color': [0, 255, 255], 'classid': 2},
 {'name': 'instrument-wrist', 'color': [125, 255, 12], 'classid': 3},
 {'name': 'kidney-parenchyma', 'color': [255, 55, 0], 'classid': 4},
 {'name': 'covered-kidney', 'color': [24, 55, 125], 'classid': 5},
 {'name': 'thread', 'color': [187, 155, 25], 'classid': 6},
 {'name': 'clamps', 'color': [0, 255, 125], 'classid': 7},
 {'name': 'suturing-needle', 'color': [255, 255, 125], 'classid': 8},
 {'name': 'suction-instrument', 'color': [123, 15, 175], 'classid': 9},
 {'name': 'small-intestine', 'color': [124, 155, 5], 'classid': 10},
 {'name': 'ultrasound-probe', 'color': [12, 255, 141], 'classid': 11}]

In [3]:
# 彩色掩码生成灰度掩码，用类号替代颜色
h = 512
w = 512
output_dir = '/home/duhj/datasets/2018-12colors/train/label/'
!mkdir -p $output_dir
for id in range(1, 17):    
    data_dir = '/home/duhj/datasets/miccai_2018/train_data/seq_' + str(id) +'/labels/'
    files = glob.iglob(data_dir+'/*.png')  
    # count = 0
    for f in files:
        image = cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (w, h), cv2.INTER_CUBIC)  # type: ignore cv2的w在h前
        # 背景白色
        mask = np.zeros([h, w, 1], np.uint8)
        threshold = 10  # 差值小于阈值的像素将被替换
        for i in range(1, len(label_list)):
            # 要替换的颜色
            target_color = np.array(label_list[i]['color']) 
            # 计算每个像素与目标颜色的差值
            color_difference = np.abs(image - target_color)
            # 计算差值的总和（BGR通道之和）
            color_difference_sum = np.sum(color_difference, axis=2)
            # 找到需要替换的像素索引
            replace_indices = np.where(color_difference_sum < threshold)
            # 将需要替换的像素值设置为掩码类别编号
            mask[replace_indices] = label_list[i]['classid']
        save_name = output_dir + 'seq_' + str(id) + '_' + f[-12:-4] +'.bmp'
        cv2.imwrite(save_name, mask)
        print("{}".format(save_name), end='\r')
        # count += 1


In [5]:
# 检查某张mask图像中，不同颜色掩码生成的不同数值的个数
checkimage = cv2.imread('/home/duhj/datasets/2018-12colors/train/label/seq_1_frame050.bmp', cv2.IMREAD_GRAYSCALE)

for k in range(12):
    count = 0
    for i in range(checkimage.shape[0]):
        for j in range(checkimage.shape[1]):
            # if checkimage[i][j] ==mask_list[k]['classid']:
            if checkimage[i][j] ==k:
                count+=1
    print(k, count)

0 106406
1 15091
2 19714
3 0
4 84684
5 0
6 3604
7 10212
8 0
9 0
10 22433
11 0


In [4]:
# 训练掩码图中，包括0在内的所有像素的值，计总数，最后的到所有类别的占比，求出各类的权重
data_dir = '/home/duhj/datasets/2018-12colors/val/label/'

files = glob.iglob(data_dir+'*.bmp')
pics = 0
counts = None
num_classes = len(label_list)
for f in files:
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    hist = cv2.calcHist([img], [0], None, [num_classes ], [0, num_classes ])
    if counts is None:
        counts = hist
    else:
        counts += hist
    # print(str(f), end='\r')
    pics += 1
print(counts)
print(pics)

[[1.2702312e+08]
 [2.2617780e+07]
 [4.9349580e+06]
 [5.6081730e+06]
 [6.0075736e+07]
 [3.0379388e+07]
 [3.2760800e+05]
 [6.2765000e+04]
 [4.7133000e+04]
 [0.0000000e+00]
 [7.9977820e+06]
 [2.2830950e+06]]
997


In [7]:
# 彩色掩码生成灰度掩码，用类号替代颜色
h = 512
w = 512
output_dir = '/home/duhj/datasets/2018-12colors/val/label/'
!mkdir -p $output_dir
for id in range(1, 17):    
    data_dir = '/home/duhj/datasets/miccai_2018/test_data/seq_' + str(id) +'/labels/'
    files = glob.iglob(data_dir+'/*.png')  
    # count = 0
    for f in files:
        image = cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (w, h), cv2.INTER_CUBIC)  # type: ignore cv2的w在h前
        # 背景白色
        mask = np.zeros([h, w, 1], np.uint8)
        threshold = 10  # 差值小于阈值的像素将被替换
        for i in range(1, len(label_list)):
            # 要替换的颜色
            target_color = np.array(label_list[i]['color']) 
            # 计算每个像素与目标颜色的差值
            color_difference = np.abs(image - target_color)
            # 计算差值的总和（BGR通道之和）
            color_difference_sum = np.sum(color_difference, axis=2)
            # 找到需要替换的像素索引
            replace_indices = np.where(color_difference_sum < threshold)
            # 将需要替换的像素值设置为掩码类别编号
            mask[replace_indices] = label_list[i]['classid']
        save_name = output_dir + 'seq_' + str(id) + '_' + f[-12:-4] +'.bmp'
        cv2.imwrite(save_name, mask)
        print("{}".format(save_name), end='\r')
        # count += 1


In [10]:
# 测试/验证 掩码图中，包括0在内的所有类的张数，计总数，最后的到所有类别的占比，求出各类的权重
data_dir = '/home/duhj/datasets/2018-12colors/val/label/'

files = glob.iglob(data_dir+'*.bmp')
pics = 0
counts = None
num_classes = len(label_list)
for f in files:
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    hist = cv2.calcHist([img], [0], None, [num_classes ], [0, num_classes ])
    if counts is None:
        counts = hist
    else:
        counts += hist
    # print(str(f), end='\r')
    pics += 1
print(counts)
print(pics)

[[1.2702312e+08]
 [2.2617780e+07]
 [4.9349580e+06]
 [5.6081730e+06]
 [6.0075736e+07]
 [3.0379388e+07]
 [3.2760800e+05]
 [6.2765000e+04]
 [4.7133000e+04]
 [0.0000000e+00]
 [7.9977820e+06]
 [2.2830950e+06]]
997


In [15]:
# 查看实际共有多少类别
data_dir = '256/mask'
files = glob.iglob(data_dir+'/*.png')
max = 0
for f in files:
    for k in range(1, len(label_list)):
        
        count = 0
        for i in range(checkimage.shape[0]):
            for j in range(checkimage.shape[1]):
                if checkimage[i][j] ==label_list[k]['classid']:
                    count+=1
                    if label_list[k]['classid'] > max:
                        max = label_list[k]['classid']
                        break
print(max)

5


In [2]:
# 把all_data下所有子目录中的图像汇总到out_images
for id in range(1, 17):
    
    data_dir = '/home/duhj/datasets/miccai_2018/seq_' + str(id) +'/left_frames/'
    files = glob.iglob(data_dir+'/*.png')

    for f in files:
        save_name = '/home/duhj/datasets/davinci_1/image/seq_'+str(id)+'_' + f[-12:-4] +'.png'
        !cp $f $save_name

In [4]:
# 把all_data下所有子目录中的图像缩放后汇总到out_images
output_dir = '/home/duhj/datasets/davinci/train/image/'
!mkdir -p $output_dir
for id in range(1, 17):
    data_dir = '/home/duhj/datasets/miccai_2018/train_data/seq_' + str(id) +'/left_frames/'
    files = glob.iglob(data_dir+'/*.png')
    count =0
    for filename in files:
        img = cv2.imread(filename, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)  # type: ignore
        savename = output_dir + 'seq_'+str(id)+'_' + filename[-12:-4]+'.bmp'
        cv2.imwrite(savename, img)
        print(filename, end='\r')
        count += 1
        if count == 22:
            break
    

In [10]:
import cv2  
import numpy as np  
import os  
  
# 设定图片所在的文件夹路径  
folder_path = '/home/duhj/ContrastiveSeg/output/davinci/hrnet_w48_9class-onlyce_t/label'  # 需要替换为你的文件夹路径  
  
# 用于保存所有的图片数据  
all_images = []  
count = 0
# 遍历文件夹下的所有文件  
for filename in os.listdir(folder_path):  
    # 检查文件是否是bmp图片  
    if filename.endswith('.png'):  
        # 读取图片  
        img = cv2.imread(os.path.join(folder_path, filename))  
        # 将图片数据转化为numpy数组并添加到列表中  
        all_images.append(np.array(img))  
    count += 1
    if count > 10: break
  
# 将所有的图片数据拼接在一起  
all_images_combined = np.concatenate(all_images, axis=0)  
print(all_images_combined.shape)
# 保存为npy文件  
# np.save('all_images.npy', all_images_combined)

(5632, 512, 3)
